# 02 Tumour segmentation using the pre-trained 'segment_glioma' model


Arvid Lundervold/Marianne Hannisdal

Last updated: **2025-03-13**  

Kernel: segment_glioma (Python 3.9.19)

In [4]:
import torch
print(torch.cuda.is_available())


True


In [5]:
!nvidia-smi

Thu Mar 13 18:31:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   45C    P8              10W /  90W |     11MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Import libraries we will use

In [6]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '18'

In [7]:
# Import libraries
import os
import os.path as op
import glob
import shutil
import subprocess as subp
import pathlib
import platform
import shutil
import IPython
from datetime import date
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pickle
import pydicom
from pydicom import dcmread
# import imageio
import imageio.v2 as imageio

%matplotlib inline
home = os.path.expanduser('~')            # To make a path to local home directory
                       
warnings.filterwarnings('ignore')  # To ignore warnings 
  

In [8]:
print(f'platform: {platform.platform()}')
print(f'python version: {platform.python_version()}')
print(f'pydicom version: {pydicom.__version__}')
print(f'nibabel version: {nib.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')


platform: Linux-6.8.0-45-generic-x86_64-with-glibc2.39
python version: 3.9.19
pydicom version: 2.4.4
nibabel version: 5.2.1
pandas version: 2.2.2
numpy version: 1.26.4


In [9]:
verbose = True

**Defining relevant directories:**

In [ ]:
PRJ_DIR = f'{home}/prj/glioma_recurrence/glioma_recurrence'
SITE = '10'
EASYREG_dir = f'{PRJ_DIR}/data/{SITE}/easyreg_1'
SEGMENT_GLIOMA_dir = f'{PRJ_DIR}/data/{SITE}/segment_glioma_'

**Define all the channels and get a list of all subjects in the directory:**

In [14]:
all_chns = ['T1', 'CT1', 'T2', 'FLAIR']
#all_chns = ['CT1']

all_dirs = sorted(glob.glob(f'{EASYREG_dir}/{SITE}_*'))    # exams from local recordings
subjs = [ os.path.basename(d) for d in all_dirs]
n = len(subjs)

print(f'{n} examinations:')
print('[')
for i, sub in enumerate(subjs):
      print(f"'{sub}',")
print(']')

1 examinations:
[
'10_036_20150101',
]


## Running PICTURES's `segment_glioma` on coregistered mpMRI images


In [19]:
!segment_glioma --help

[13:39:39] INFO     platform: linux                            ]8;id=355899;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py\doInference.py]8;;\:]8;id=333569;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py#14\14]8;;\
[13:39:40] INFO     Found GPU, using cuda device 0: NVIDIA RTX ]8;id=22763;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py\doInference.py]8;;\:]8;id=826317;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py#19\19]8;;\
                    3500 Ada Generation Laptop GPU                              
                                                                                
 Usage: segment_glioma [OPTIONS] T1CE                                           
                              

In [19]:
# Execute tree command on the easyreg subdirecory 
print(os.popen(f'{TREE} {EASYREG_dir}').read())

/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/easyreg_1
└── 10_036_20150101

1 directory, 0 files



## Preoperative segment_glioma model

In [20]:
%%bash -s "$EASYREG_dir" "$SEGMENT_GLIOMA_dir" 

list=(
    '10_036_20150101'
)


for i in "${!list[@]}"; do
    list_new[$i]=${list[$i]//,/}
done

for element in "${list_new[@]}"; do
    echo "RUNNING EXAM: $element"

    NIFTI_COREG_DIR=$1/${element}
    SEGMENT_GLIOMA_DIR=$2/${element}    
    mkdir -p $SEGMENT_GLIOMA_DIR

    segment_glioma   $NIFTI_COREG_DIR/${element}_CT1_coreg.nii.gz \
    --t1 $NIFTI_COREG_DIR/${element}_T1_coreg.nii.gz \
    --t2 $NIFTI_COREG_DIR/${element}_T2_coreg.nii.gz \
    --flair $NIFTI_COREG_DIR/${element}_FLAIR_coreg.nii.gz \
    --sessiontype preop

    # mv results to the SEGMENT_GLIOMA_DIR
    cp $NIFTI_COREG_DIR/${element}_CT1_coreg.nii.gz $SEGMENT_GLIOMA_DIR/
    mv $NIFTI_COREG_DIR/brain*coreg.nii.gz $SEGMENT_GLIOMA_DIR/
    mv $NIFTI_COREG_DIR/segment*coreg.nii.gz $SEGMENT_GLIOMA_DIR/
    cp -r $NIFTI_COREG_DIR/wdir $SEGMENT_GLIOMA_DIR/
    rm -r $NIFTI_COREG_DIR/wdir
done


RUNNING EXAM: 10_036_20150101
[18:35:02] INFO     platform: linux                            ]8;id=611031;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py\doInference.py]8;;\:]8;id=675592;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py#14\14]8;;\
           INFO     Found GPU, using cuda device 0: NVIDIA RTX ]8;id=8916;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py\doInference.py]8;;\:]8;id=985666;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py#19\19]8;;\
                    3500 Ada Generation Laptop GPU                              
[18:35:06] INFO     Running preop segmentation model          ]8;id=100779;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture

/home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/HD_BET/run.py:75: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



File: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/easyreg_1/10_036_20150101/wdir/tmpw6rg02cd/nii_data_tki/10_036_20150101_CT1_coreg.nii.gz
preprocessing...
image shape after preprocessing:  (121, 145, 121)
prediction (CNN id)...
0
1
2
3
4
exporting segmentation...
mask_fname: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/easyreg_1/10_036_20150101/wdir/tmpw6rg02cd/nii_data_tki/brain_mask_10_036_20150101_CT1_coreg_mask.nii.gz
[18:35:40] INFO     5&6/10) Apply skullstrip and              ]8;id=529322;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py\doInference.py]8;;\:]8;id=258851;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py#157\157]8;;\
                    normalization                                               
[18:35:41] INFO     07/10) Load segmentation network          ]8;id=95676;file:///home/ma

/home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/nnunet/training/model_restore.py:147: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



starting preprocessing generator
starting prediction...
preprocessing /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/easyreg_1/10_036_20150101/wdir/tmpw6rg02cd/nnUNet_workdir/output/tmp.nii.gz
using preprocessor GenericPreprocessor
before crop: (4, 182, 218, 182) after crop: (4, 182, 218, 182) spacing: [1. 1. 1.] 

no resampling necessary
no resampling necessary
before: {'spacing': array([1., 1., 1.]), 'spacing_transposed': array([1., 1., 1.]), 'data.shape (data is transposed)': (4, 182, 218, 182)} 
after:  {'spacing': array([1., 1., 1.]), 'data.shape (data is resampled)': (4, 182, 218, 182)} 

(4, 182, 218, 182)
This worker has ended successfully, no errors to report
predicting /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/easyreg_1/10_036_20150101/wdir/tmpw6rg02cd/nnUNet_workdir/output/tmp.nii.gz


/home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/nnunet/training/network_training/network_trainer.py:404: FutureWarning:

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.



debug: mirroring True mirror_axes (0, 1, 2)


/home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/nnunet/network_architecture/neural_network.py:141: FutureWarning:

`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.



step_size: 0.5
do mirror: True
data shape: (4, 182, 218, 182)
patch size: [128 160 112]
steps (x, y, and z): [[0, 54], [0, 58], [0, 35, 70]]
number of tiles: 12
computing Gaussian
done
prediction done
inference done. Now waiting for the segmentation export to finish...
force_separate_z: None interpolation order: 1
no resampling necessary
WARNING! Cannot run postprocessing because the postprocessing file is missing. Make sure to run consolidate_folds in the output folder of the model first!
The folder you need to run this in is /home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/./model/Task101_picture/
[18:35:51] INFO     DONE!!! Saved segmentations to:           ]8;id=101158;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.py\doInference.py]8;;\:]8;id=338188;file:///home/marianne/anaconda3/envs/segment_glioma/lib/python3.9/site-packages/picture_nnunet_package/doInference.